## Kernel: Python 3.8

In [1]:
from dotenv import dotenv_values
from datetime import timedelta, datetime

import pandas as pd
import requests
import locale

## Get Authority

In [4]:
config = dotenv_values(".env") # get values from .env file
locale.setlocale(locale.LC_ALL, "id") # set local language

def login() -> str:
    try:
        url = 'https://api.kampusmerdeka.kemdikbud.go.id/user/auth/login/mbkm'
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
        }
        account = {
            'email': config.get('EMAIL'),
            'password': config.get('PASSWORD')
        }
        response = requests.post(url, json=account, headers=headers)
        token = response.json()['data'].get('access_token')
    
    except Exception:
        raise ValueError(response.json()["error"]["message"])
    
    return token, account

(token, account) = login()
headers = dict()
headers["Accept"] = 'application/json'
headers["Authorization"] = f'Bearer {token}'
headers["User-Agent"] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'

print(f'login to {account["email"]} success')

login to hazlanqodri2020work@gmail.com success


## 1. Fetch ALL Mitra (Raw & Test)

In [ ]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=20&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
reports = response.json().get('data')

print(reports)

## 2. Process Data

In [19]:
url = f'https://api.kampusmerdeka.kemdikbud.go.id/magang/browse/position?offset=0&limit=10000&location_key=&mitra_key=&keyword=&sector_id='
response = requests.get(url, headers=headers)
mitra_data = response.json().get('data')

id_cols = []
name_cols = []
activity_name_cols = []
quotas_cols = []

for mitra_data_detail in mitra_data:
    # fetch data from mitra_data_detail dict in mitra_data list
    id = mitra_data_detail["id"]
    name = mitra_data_detail["name"]
    activity_name = mitra_data_detail["activity_name"]
    quotas = mitra_data_detail["total"]

    # Append Data
    id_cols.append(id)
    name_cols.append(name)
    activity_name_cols.append(activity_name)
    quotas_cols.append(quotas)

mitra_data_df = pd.DataFrame({
    "id": id_cols,
    "name": name_cols,
    "activity_name": activity_name_cols,
    "quotas": quotas_cols
})


In [20]:
mitra_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710 entries, 0 to 709
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             710 non-null    object
 1   name           710 non-null    object
 2   activity_name  710 non-null    object
 3   quotas         710 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 22.3+ KB


In [18]:
mitra_data_df

,id,name,activity_name,quotas
0,7659b95a-6429-11ed-b5cc-0a223dcf0f66,Academic Officers Intern,Zenius Professionals Practice Program Angkatan 4,6
1,3f356ec2-6fb0-11ed-a7a1-0278cb7f6574,Accounting Improvement,Great ISS Internship Program (GETS Program),2
2,99c6bc1f-5c0d-11ed-8806-7a77835150ba,Accounting Intern,[MMN-JTSE] Pengembangan Bisnis Perusahaan mela...,1
3,c53f879e-6a30-11ed-8ebe-a6bfdab34a31,Accounting Officer,Officer Development Program,2
4,02aaaa53-60b4-11ed-aaab-4a119f027d25,Accounting Officer - Jakarta (Head Office),ASTERIEX IV : First Journey to Become ASSA SPE...,2
...,...,...,...,...
704,3058ed45-6f9b-11ed-83fd-7e8ddab09462,Workshop Quality & Productivity Control,Belajar Birdsama Bluebird,4
705,947ba8dc-5502-11ed-b209-b250481d78c3,Youth Segment Intern,DYNAMIC SUSTAINABILITY PROGRAMS (OFFLINE INTER...,4
706,67b505b2-6b0c-11ed-8225-d61b0e2692e7,Testing Specialist - Custody & FA System,Custody & Fund Admin System,3
707,f52a4546-6b0c-11ed-a22e-c63f61abb566,Treasury Digital Interactive Communication Pr...,Treasury Digital Interactive Communication,3
